In [120]:
import numpy as np
import pandas as pd
import os
from itertools import product
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf

from statsmodels.tsa.stattools import adfuller

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error


dataset=pd.read_csv("/Users/alishadaredia/Desktop/Ryerson - Arif/CIND820/Price dataset/BaseMetalPricePrediction.csv")
dataset.describe()
dataset.head()

,Date,Price,Open,High,Low,Vol.,Change %,Commodity,US CPI,China CPI,US PMI,China PMI
0,2023-08-31,"25,396.00","25,300.00","25,150.00","25,110.00",1.25K,-0.31%,Tin,140.801768,103.2,47.6,49.7
1,2023-08-30,"25,475.00","25,400.00","25,300.00","25,325.00",1.27K,0.32%,Tin,140.801768,103.2,47.6,49.7
2,2023-08-29,"25,395.00","25,200.00","25,050.00","25,055.00",1.32K,-0.40%,Tin,140.801768,103.2,47.6,49.7
3,2023-08-25,"25,497.00","25,600.00","25,500.00","25,575.00",0.59K,-1.44%,Tin,140.801768,103.2,47.6,49.7
4,2023-08-24,"25,870.00","25,900.00","25,825.00","25,850.00",0.39K,-0.98%,Tin,140.801768,103.2,47.6,49.7


In [121]:
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14928 entries, 0 to 14927
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       14928 non-null  object 
 1   Price      14928 non-null  object 
 2   Open       14928 non-null  object 
 3   High       14928 non-null  object 
 4   Low        14928 non-null  object 
 5   Vol.       7661 non-null   object 
 6   Change %   14928 non-null  object 
 7   Commodity  14928 non-null  object 
 8   US CPI     14928 non-null  float64
 9   China CPI  14928 non-null  float64
 10  US PMI     14928 non-null  float64
 11  China PMI  14928 non-null  float64
dtypes: float64(4), object(8)
memory usage: 1.4+ MB


In [122]:
dataset["Date"] = dataset["Date"].astype('datetime64[ns]')
dataset["Price"] = dataset["Price"].str.replace(",", "")
dataset["Open"] = dataset["Open"].str.replace(",", "")
dataset["High"] = dataset["High"].str.replace(",", "")
dataset["Low"] = dataset["Low"].str.replace(",", "")

dataset["Price"] = dataset["Price"].astype('float64')
dataset["Open"] = dataset["Open"].astype('float64')
dataset["High"] = dataset["High"].astype('float64')
dataset["Low"] = dataset["Low"].astype('float64')
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14928 entries, 0 to 14927
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       14928 non-null  datetime64[ns]
 1   Price      14928 non-null  float64       
 2   Open       14928 non-null  float64       
 3   High       14928 non-null  float64       
 4   Low        14928 non-null  float64       
 5   Vol.       7661 non-null   object        
 6   Change %   14928 non-null  object        
 7   Commodity  14928 non-null  object        
 8   US CPI     14928 non-null  float64       
 9   China CPI  14928 non-null  float64       
 10  US PMI     14928 non-null  float64       
 11  China PMI  14928 non-null  float64       
dtypes: datetime64[ns](1), float64(8), object(3)
memory usage: 1.4+ MB


In [123]:
dataset.describe()

,Price,Open,High,Low,US CPI,China CPI,US PMI,China PMI
count,14928.000000,14928.000000,14928.000000,14928.000000,14928.000000,14928.000000,14928.000000,14928.000000
mean,11675.353999,11675.556693,11738.159450,11609.762911,112.731793,90.784734,53.440287,50.652693
std,8620.380610,8627.157952,8649.762122,8593.869623,10.922922,8.582517,5.496316,2.249782
min,848.000000,848.000000,848.000000,848.000000,96.410318,74.960150,32.400000,35.700000
25%,2868.000000,2868.000000,2868.000000,2868.000000,105.392893,84.520710,50.600000,50.000000
50%,9520.000000,9525.250000,9555.500000,9499.000000,109.724818,91.083430,53.600000,50.600000
75%,18405.500000,18394.000000,18510.625000,18301.250000,117.847979,98.972540,57.300000,51.500000
max,81051.500000,81066.500000,86791.500000,81031.500000,140.801768,104.000000,64.700000,56.600000


In [124]:
#Printing the unique values for metals

for col in ["Commodity"]:
    print(f"Unique values for {col}: {dataset[col].unique()}")

Unique values for Commodity: ['Tin' 'Nickel' 'Lead' 'Copper']


In [125]:
#Plotting correlation for the dataframe

import matplotlib
matplotlib.use('TkAgg')
sns.heatmap(dataset.corr(numeric_only=True))
plt.show()

In [77]:
#Using pandas profiling for EDA

import pandas_profiling

# Generate profile report
profile = pandas_profiling.ProfileReport(dataset)

# Output report to HTML file
profile.to_file("MetalPrice_EDAoutput.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [126]:
#Plotting distribution of close price for each metal

for name, group in dataset.groupby('Commodity'):
    plt.hist(group['Price'], label=name)
    
plt.show()

In [127]:
# Plotting distribution of close price for each metal - Part 2

fig, ax = plt.subplots()

# Iterate through groups and plot histograms
for name, group in dataset.groupby('Commodity'):
    ax.hist(group['Price'], label=name, alpha=0.5)  

# Add axis titles and labels
ax.set_xlabel('Close Price')
ax.set_ylabel('Frequency')

# Add a legend
ax.legend()

plt.show()


In [130]:
#Check the duplicates
dataset.duplicated().sum()

0

In [131]:
#Check the null values

dataset.isnull().sum()

Date            0
Price           0
Open            0
High            0
Low             0
Vol.         7267
Change %        0
Commodity       0
US CPI          0
China CPI       0
US PMI          0
China PMI       0
dtype: int64

In [132]:
#Plotting correlation matrix with labels

plt.figure(figsize=(15,10))
sns.heatmap(dataset.corr(numeric_only=True), annot = True)

plt.show()

In [31]:
# Convert 'Date' column to datetime and set it as the index
# dataset['Date'] = pd.to_datetime(dataset['Date'])
# dataset.set_index('Date', inplace=True)

# # Resample to a monthly frequency, considering only values on the 1st of every month
# monthly_cpi = dataset['US CPI'].resample('MS').mean()

# # Create a line plot for the trend
# plt.figure(figsize=(10, 6))
# plt.plot(monthly_cpi.index, monthly_cpi, marker='o', linestyle='-')
# plt.title('US CPI Trend (Monthly)')
# plt.xlabel('Date')
# plt.ylabel('US CPI')
# plt.grid(True)

# plt.show()

In [32]:
# # Convert 'Date' column to datetime and set it as the index
# #dataset['Date'] = pd.to_datetime(dataset['Date'])
# #dataset.set_index('Date', inplace=True)

# # Resample to a monthly frequency, considering only values on the 1st of every month
# monthly_cpi_China = dataset['China CPI'].resample('MS').mean()

# # Create a line plot for the trend
# plt.figure(figsize=(10, 6))
# plt.plot(monthly_cpi_China.index, monthly_cpi_China, marker='o', linestyle='-')
# plt.title('China CPI Trend (Monthly)')
# plt.xlabel('Date')
# plt.ylabel('China CPI')
# plt.grid(True)

# plt.show()

In [133]:
#Our goal will be to predict the stock prices of the different commodities in 2022-23 using the previous stock 
#data. To do this, we will need to add some features to our dataset before passing it into modelling:
#We will add seasonality data captured by day, week, month, day of week, etc.

# add datetime specific features
dataset['timestamp'] = dataset['Date'].apply(lambda d: d.timestamp())
dataset['day'] = dataset['Date'].dt.day
dataset['week'] = dataset['Date'].dt.isocalendar().week
dataset['month'] = dataset['Date'].dt.month
dataset['year'] = dataset['Date'].dt.year
dataset['day_of_week'] = dataset['Date'].dt.dayofweek

dataset.sample(5)

,Date,Price,Open,High,Low,Vol.,Change %,Commodity,US CPI,China CPI,US PMI,China PMI,timestamp,day,week,month,year,day_of_week
8782,2018-06-11,2484.0,2475.75,2504.75,2473.25,NaN,0.50%,Lead,115.561864,94.04185,60.2,51.5,1.528675e+09,11,24,6,2018,0
5146,2018-01-22,12735.0,12685.00,12840.00,12647.50,NaN,0.37%,Nickel,113.671519,94.31080,59.1,51.3,1.516579e+09,22,4,1,2018,0
11519,2022-05-18,9235.0,9265.00,9260.00,9260.00,26.35K,-1.40%,Copper,134.046607,102.70000,56.1,49.6,1.652832e+09,18,20,5,2022,2
11588,2022-02-07,9777.5,9820.00,9814.50,9815.00,20.42K,-0.65%,Copper,130.111829,102.50000,58.6,50.2,1.644192e+09,7,6,2,2022,0
189,2022-11-28,22354.0,22300.00,22250.00,22300.00,0.66K,0.55%,Tin,136.529920,103.30000,49.0,48.0,1.669594e+09,28,48,11,2022,0


In [134]:
commodities = ["Nickel", "Tin", "Lead", "Copper"]

In [135]:
#We will drop Volume as there are more than 50% of them as blanks and should not be a part of modelling
#Secondly, we are dropping other prices like open, High, Low as they have a high correlation 
#and it does not make sense to predict close price based on price data from the same day 

dataset.drop(['Vol.', 'Change %', 'Open', 'High','Low'], axis=1, inplace=True)
dataset.sample(5)

,Date,Price,Commodity,US CPI,China CPI,US PMI,China PMI,timestamp,day,week,month,year,day_of_week
9202,2016-10-10,2102.0,Lead,110.856640,91.71098,51.9,51.2,1.476058e+09,10,41,10,2016,0
14464,2010-07-14,6706.0,Copper,99.979592,77.30582,55.5,51.2,1.279066e+09,14,28,7,2010,2
1344,2018-05-04,21217.5,Tin,115.377966,94.04185,58.7,51.9,1.525392e+09,4,18,5,2018,4
4648,2020-01-10,14190.0,Nickel,118.305202,101.12410,50.9,50.0,1.578614e+09,10,2,1,2020,4
3124,2011-03-02,31582.0,Tin,102.481708,81.71854,61.2,53.4,1.299024e+09,2,9,3,2011,2


In [136]:
#Checking the structure of the dataset after dropping the extra columns
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14928 entries, 0 to 14927
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         14928 non-null  datetime64[ns]
 1   Price        14928 non-null  float64       
 2   Commodity    14928 non-null  object        
 3   US CPI       14928 non-null  float64       
 4   China CPI    14928 non-null  float64       
 5   US PMI       14928 non-null  float64       
 6   China PMI    14928 non-null  float64       
 7   timestamp    14928 non-null  float64       
 8   day          14928 non-null  int64         
 9   week         14928 non-null  UInt32        
 10  month        14928 non-null  int64         
 11  year         14928 non-null  int64         
 12  day_of_week  14928 non-null  int64         
dtypes: UInt32(1), datetime64[ns](1), float64(6), int64(4), object(1)
memory usage: 1.4+ MB


In [137]:
#Right now, the data for each metal is stacked one on another and it will be easier 
#to analyse if we have them side by side based on date and hence we will pivot the data

pivoted_dataset = dataset.pivot(index='Date', columns='Commodity', values='Price')

# Reset the index and remove the column names
pivoted_dataset.reset_index(inplace=True)
pivoted_dataset.columns.name = None

# Merge with the original DataFrame using 'Date' as the key
pivoted_dataset = pd.merge(dataset[['Date', 'US CPI', 'China CPI', 'US PMI', 'China PMI', 'timestamp', 'day', 'week', 'month', 'year', 'day_of_week']], pivoted_dataset, on='Date')

# Rename the columns for clarity
pivoted_dataset.rename(columns={'Tin': 'Tin_Price', 'Copper': 'Copper_Price', 'Nickel': 'Nickel_Price', 'Lead': 'Lead_Price'}, inplace=True)

# Display a sample of resulting DataFrame
pivoted_dataset.sample(5)

,Date,US CPI,China CPI,US PMI,China PMI,timestamp,day,week,month,year,day_of_week,Copper_Price,Lead_Price,Nickel_Price,Tin_Price
2587,2021-02-04,120.617916,101.50000,60.8,50.6,1.612397e+09,4,5,2,2021,3,7823.00,2044.00,17646.0,22825.0
11322,2012-06-15,105.238345,83.81767,49.7,50.2,1.339718e+09,15,24,6,2012,4,7499.25,1911.50,16791.0,19737.0
11490,2012-04-05,105.516715,84.52071,54.8,53.3,1.333584e+09,5,14,4,2012,3,8381.00,2061.00,18342.0,23123.0
4103,2019-08-07,117.657202,97.62781,49.1,49.5,1.565136e+09,7,32,8,2019,2,5745.00,2018.75,15460.0,16750.0
498,2023-03-02,138.421640,103.20000,46.3,51.9,1.677715e+09,2,9,3,2023,3,8958.50,2125.50,24395.0,24570.0


In [138]:
#Next we will create 3 datasets for training (2008-2016), validation (2017-21) and test set (2022-23)
# Filter the records for training, validation, and test datasets based on the date ranges
training_df = pivoted_dataset[pivoted_dataset['Date'] <= '2016-12-31']
validation_df = pivoted_dataset[(pivoted_dataset['Date'] >= '2017-01-01') & (pivoted_dataset['Date'] <= '2021-12-31')]
test_df = pivoted_dataset[pivoted_dataset['Date'] >= '2022-01-01']

# Reset the index for each dataset
training_df.reset_index(drop=True, inplace=True)
validation_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

# Display the resulting datasets
print("Training Dataset:")
print(training_df)

# print("\nValidation Dataset:")
# print(validation_df)

# print("\nTest Dataset:")
# print(test_df)

Training Dataset:
           Date      US CPI  China CPI  US PMI  China PMI     timestamp  day  \
0    2016-12-30  110.720436   91.97992    54.7       51.4  1.483056e+09   30   
1    2016-12-30  110.720436   91.97992    54.7       51.4  1.483056e+09   30   
2    2016-12-30  110.720436   91.97992    54.7       51.4  1.483056e+09   30   
3    2016-12-30  110.720436   91.97992    54.7       51.4  1.483056e+09   30   
4    2016-12-29  110.720436   91.97992    54.7       51.4  1.482970e+09   29   
...         ...         ...        ...     ...        ...           ...  ...   
8195 2008-07-08  100.875236   76.32951    50.0       48.4  1.215475e+09    8   
8196 2008-07-07  100.875236   76.32951    50.0       48.4  1.215389e+09    7   
8197 2008-07-07  100.875236   76.32951    50.0       48.4  1.215389e+09    7   
8198 2008-07-07  100.875236   76.32951    50.0       48.4  1.215389e+09    7   
8199 2008-07-07  100.875236   76.32951    50.0       48.4  1.215389e+09    7   

      week  month  ye

In [139]:
print("\nTest Dataset:")
print(test_df)


Test Dataset:
           Date      US CPI  China CPI  US PMI  China PMI     timestamp  day  \
0    2023-08-31  140.801768      103.2    47.6       49.7  1.693440e+09   31   
1    2023-08-31  140.801768      103.2    47.6       49.7  1.693440e+09   31   
2    2023-08-31  140.801768      103.2    47.6       49.7  1.693440e+09   31   
3    2023-08-31  140.801768      103.2    47.6       49.7  1.693440e+09   31   
4    2023-08-30  140.801768      103.2    47.6       49.7  1.693354e+09   30   
...         ...         ...        ...     ...        ...           ...  ...   
1663 2022-01-05  128.934147      101.9    57.6       50.1  1.641341e+09    5   
1664 2022-01-04  128.934147      101.9    57.6       50.1  1.641254e+09    4   
1665 2022-01-04  128.934147      101.9    57.6       50.1  1.641254e+09    4   
1666 2022-01-04  128.934147      101.9    57.6       50.1  1.641254e+09    4   
1667 2022-01-04  128.934147      101.9    57.6       50.1  1.641254e+09    4   

      week  month  year 

In [143]:
#Next we will apply the 3 models starting with Random Forest
# We will print the key metrcis like R-squared,Inference Time, RMSE 
#and plot the actual vs. predicted stocks price for each metal

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import time
import matplotlib.pyplot as plt

# Define the metals
metals = ['Tin_Price', 'Copper_Price', 'Nickel_Price', 'Lead_Price']

# Define a function to train, evaluate, and plot for each metal
def train_evaluate_plot(metal):
    print(f"Modeling for {metal}...")

    # Select the relevant columns for the metal
    X_train = training_df[['US CPI', 'China CPI', 'US PMI', 'China PMI', 'day', 'week', 'month', 'year']]
    y_train = training_df[metal]
    X_val = validation_df[['US CPI', 'China CPI', 'US PMI', 'China PMI', 'day', 'week', 'month', 'year']]
    y_val = validation_df[metal]
    X_test = test_df[['US CPI', 'China CPI', 'US PMI', 'China PMI', 'day', 'week', 'month', 'year']]
    y_test = test_df[metal]

    # Create a Random Forest Regressor model
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

    # Train the model on the training set
    start_time = time.time()
    rf_model.fit(X_train, y_train)
    training_time = time.time() - start_time

    # Make predictions on the validation set
    y_val_pred = rf_model.predict(X_val)

    # Calculate metrics for validation
    mse = mean_squared_error(y_val, y_val_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_val, y_val_pred)

    # Print validation metrics
    print(f"Validation Metrics for {metal}:")
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    print(f"R-squared (R2): {r2}")
    print(f"Inference Time (Validation): {training_time:.4f} seconds")

    # Make predictions on the test set
    start_time = time.time()
    y_test_pred = rf_model.predict(X_test)
    inference_time = time.time() - start_time

    # Print inference time on the test set
    print(f"Inference Time (Test): {inference_time:.4f} seconds")

    # Create a time series plot for predicted and actual prices
    plt.figure(figsize=(10, 6))
    plt.plot(test_df['Date'], y_test, label="Actual", marker='o', markersize=2)
    plt.plot(test_df['Date'], y_test_pred, label="Predicted", marker='o', markersize=2)
    plt.title(f"{metal} Price Prediction (Test Set)")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.grid(True)
    plt.show()

# Iterate over each metal to train, evaluate, and plot
for metal in metals:
    train_evaluate_plot(metal)



Modeling for Tin_Price...
Validation Metrics for Tin_Price:
Mean Squared Error (MSE): 45513308.0430074
Root Mean Squared Error (RMSE): 6746.355167274208
R-squared (R2): -0.39951630912061065
Inference Time (Validation): 0.9708 seconds
Inference Time (Test): 0.0098 seconds


2023-11-04 00:51:15.898 python[39388:23371058] +[CATransaction synchronize] called within transaction


Modeling for Copper_Price...
Validation Metrics for Copper_Price:
Mean Squared Error (MSE): 5535775.056155336
Root Mean Squared Error (RMSE): 2352.8227846897726
R-squared (R2): -2.040496254154668
Inference Time (Validation): 0.8609 seconds
Inference Time (Test): 0.0108 seconds


2023-11-04 00:52:31.890 python[39388:23371058] +[CATransaction synchronize] called within transaction


Modeling for Nickel_Price...
Validation Metrics for Nickel_Price:
Mean Squared Error (MSE): 27373155.0060331
Root Mean Squared Error (RMSE): 5231.936066699697
R-squared (R2): -2.128183571064108
Inference Time (Validation): 0.8744 seconds
Inference Time (Test): 0.0111 seconds


2023-11-04 00:52:53.980 python[39388:23371058] +[CATransaction synchronize] called within transaction
2023-11-04 00:53:06.264 python[39388:23371058] Bad cursor rect event, flags = 0


Modeling for Lead_Price...
Validation Metrics for Lead_Price:
Mean Squared Error (MSE): 127132.5070239476
Root Mean Squared Error (RMSE): 356.5564569937664
R-squared (R2): -1.3503058249253743
Inference Time (Validation): 0.8716 seconds
Inference Time (Test): 0.0101 seconds


2023-11-04 00:53:12.526 python[39388:23371058] +[CATransaction synchronize] called within transaction


In [144]:
# Secondly, using XGBoost we will print the key metrcis like R-squared,Inference Time, RMSE 
#and plot the actual vs. predicted stocks price for each metal

!pip3 install xgboost
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import time
import matplotlib.pyplot as plt

# Define the metals
metals = ['Tin_Price', 'Copper_Price', 'Nickel_Price', 'Lead_Price']

# Define a function to train, evaluate, and plot for each metal using XGBoost
def train_evaluate_plot_xgboost(metal):
    print(f"Modeling for {metal} using XGBoost...")

    # Select the relevant columns for the metal
    X_train = training_df[['US CPI', 'China CPI', 'US PMI', 'China PMI', 'day', 'week', 'month', 'year']]
    y_train = training_df[metal]
    X_val = validation_df[['US CPI', 'China CPI', 'US PMI', 'China PMI', 'day', 'week', 'month', 'year']]
    y_val = validation_df[metal]
    X_test = test_df[['US CPI', 'China CPI', 'US PMI', 'China PMI', 'day', 'week', 'month', 'year']]
    y_test = test_df[metal]

    # Create an XGBoost Regressor model
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

    # Train the model on the training set
    start_time = time.time()
    xgb_model.fit(X_train, y_train)
    training_time = time.time() - start_time

    # Make predictions on the validation set
    y_val_pred = xgb_model.predict(X_val)

    # Calculate metrics for validation
    mse = mean_squared_error(y_val, y_val_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_val, y_val_pred)

    # Print validation metrics
    print(f"Validation Metrics for {metal}:")
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    print(f"R-squared (R2): {r2}")
    print(f"Inference Time (Validation): {training_time:.4f} seconds")

    # Make predictions on the test set
    start_time = time.time()
    y_test_pred = xgb_model.predict(X_test)
    inference_time = time.time() - start_time

    # Print inference time on the test set
    print(f"Inference Time (Test): {inference_time:.4f} seconds")

    # Create a time series plot for predicted and actual prices
    plt.figure(figsize=(10, 6))
    plt.plot(test_df['Date'], y_test, label="Actual", marker='o', markersize=2)
    plt.plot(test_df['Date'], y_test_pred, label="Predicted", marker='o', markersize=2)
    plt.title(f"{metal} Price Prediction using XGBoost (Test Set)")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.grid(True)
    plt.show()

# Iterate over each metal to train, evaluate, and plot using XGBoost
for metal in metals:
    train_evaluate_plot_xgboost(metal)


Modeling for Tin_Price using XGBoost...
Validation Metrics for Tin_Price:
Mean Squared Error (MSE): 38211648.69481821
Root Mean Squared Error (RMSE): 6181.557141596138
R-squared (R2): -0.17499315796276504
Inference Time (Validation): 0.1199 seconds
Inference Time (Test): 0.0030 seconds


2023-11-04 00:53:43.648 python[39388:23371058] +[CATransaction synchronize] called within transaction


Modeling for Copper_Price using XGBoost...
Validation Metrics for Copper_Price:
Mean Squared Error (MSE): 5575593.0248770295
Root Mean Squared Error (RMSE): 2361.2693672846876
R-squared (R2): -2.062366070669655
Inference Time (Validation): 0.0802 seconds
Inference Time (Test): 0.0027 seconds


2023-11-04 00:54:09.957 python[39388:23371058] +[CATransaction synchronize] called within transaction


Modeling for Nickel_Price using XGBoost...
Validation Metrics for Nickel_Price:
Mean Squared Error (MSE): 26575526.88133316
Root Mean Squared Error (RMSE): 5155.145670234078
R-squared (R2): -2.0370312287434986
Inference Time (Validation): 0.0793 seconds
Inference Time (Test): 0.0024 seconds


2023-11-04 00:54:36.702 python[39388:23371058] +[CATransaction synchronize] called within transaction


Modeling for Lead_Price using XGBoost...
Validation Metrics for Lead_Price:
Mean Squared Error (MSE): 96359.68915761942
Root Mean Squared Error (RMSE): 310.4185708968125
R-squared (R2): -0.7814070060971183
Inference Time (Validation): 0.0837 seconds
Inference Time (Test): 0.0027 seconds


2023-11-04 00:54:54.730 python[39388:23371058] +[CATransaction synchronize] called within transaction
2023-11-04 00:55:02.038 python[39388:23371058] +[CATransaction synchronize] called within transaction
2023-11-04 00:55:02.137 python[39388:23371058] +[CATransaction synchronize] called within transaction
2023-11-04 00:55:02.255 python[39388:23371058] +[CATransaction synchronize] called within transaction
2023-11-04 00:55:02.589 python[39388:23371058] +[CATransaction synchronize] called within transaction
2023-11-04 00:55:02.690 python[39388:23371058] +[CATransaction synchronize] called within transaction
2023-11-04 00:55:02.792 python[39388:23371058] +[CATransaction synchronize] called within transaction
2023-11-04 00:55:02.907 python[39388:23371058] +[CATransaction synchronize] called within transaction
2023-11-04 00:55:03.002 python[39388:23371058] Warning: Window move completed without beginning


In [145]:
# Lastly, using KNN regressor we will print the key metrcis like R-squared,Inference Time, RMSE 
#and plot the actual vs. predicted stocks price for each metal


from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import time
import matplotlib.pyplot as plt

# Define the metals
metals = ['Tin_Price', 'Copper_Price', 'Nickel_Price', 'Lead_Price']

# Define a function to train, evaluate, and plot for each metal using KNN
def train_evaluate_plot_knn(metal):
    print(f"Modeling for {metal} using K-Nearest Neighbors...")

    # Select the relevant columns for the metal
    X_train = training_df[['US CPI', 'China CPI', 'US PMI', 'China PMI', 'day', 'week', 'month', 'year']]
    y_train = training_df[metal]
    X_val = validation_df[['US CPI', 'China CPI', 'US PMI', 'China PMI', 'day', 'week', 'month', 'year']]
    y_val = validation_df[metal]
    X_test = test_df[['US CPI', 'China CPI', 'US PMI', 'China PMI', 'day', 'week', 'month', 'year']]
    y_test = test_df[metal]

    # Create a K-Nearest Neighbors Regressor model
    knn_model = KNeighborsRegressor(n_neighbors=5)

    # Train the model on the training set
    start_time = time.time()
    knn_model.fit(X_train, y_train)
    training_time = time.time() - start_time

    # Make predictions on the validation set
    y_val_pred = knn_model.predict(X_val)

    # Calculate metrics for validation
    mse = mean_squared_error(y_val, y_val_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_val, y_val_pred)

    # Print validation metrics
    print(f"Validation Metrics for {metal}:")
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    print(f"R-squared (R2): {r2}")
    print(f"Inference Time (Validation): {training_time:.4f} seconds")

    # Make predictions on the test set
    start_time = time.time()
    y_test_pred = knn_model.predict(X_test)
    inference_time = time.time() - start_time

    # Print inference time on the test set
    print(f"Inference Time (Test): {inference_time:.4f} seconds")

    # Create a time series plot for predicted and actual prices
    plt.figure(figsize=(10, 6))
    plt.plot(test_df['Date'], y_test, label="Actual", marker='o', markersize=2)
    plt.plot(test_df['Date'], y_test_pred, label="Predicted", marker='o', markersize=2)
    plt.title(f"{metal} Price Prediction using K-Nearest Neighbors (Test Set)")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.grid(True)
    plt.show()

# Iterate over each metal to train, evaluate, and plot using K-Nearest Neighbors
for metal in metals:
    train_evaluate_plot_knn(metal)


Modeling for Tin_Price using K-Nearest Neighbors...
Validation Metrics for Tin_Price:
Mean Squared Error (MSE): 42666766.00540711
Root Mean Squared Error (RMSE): 6531.980251455688
R-squared (R2): -0.3119862618110505
Inference Time (Validation): 0.0246 seconds
Inference Time (Test): 0.0151 seconds


2023-11-04 00:55:36.767 python[39388:23371058] +[CATransaction synchronize] called within transaction


Modeling for Copper_Price using K-Nearest Neighbors...
Validation Metrics for Copper_Price:
Mean Squared Error (MSE): 5248534.985332016
Root Mean Squared Error (RMSE): 2290.9681327622206
R-squared (R2): -1.8827311082587332
Inference Time (Validation): 0.0071 seconds
Inference Time (Test): 0.0147 seconds


2023-11-04 00:55:48.868 python[39388:23371058] +[CATransaction synchronize] called within transaction


Modeling for Nickel_Price using K-Nearest Neighbors...
Validation Metrics for Nickel_Price:
Mean Squared Error (MSE): 29850905.747565217
Root Mean Squared Error (RMSE): 5463.598241778509
R-squared (R2): -2.4113390626815105
Inference Time (Validation): 0.0053 seconds
Inference Time (Test): 0.0147 seconds


2023-11-04 00:56:12.344 python[39388:23371058] +[CATransaction synchronize] called within transaction


Modeling for Lead_Price using K-Nearest Neighbors...
Validation Metrics for Lead_Price:
Mean Squared Error (MSE): 124097.35157114625
Root Mean Squared Error (RMSE): 352.2745400552618
R-squared (R2): -1.2941947349511218
Inference Time (Validation): 0.0064 seconds
Inference Time (Test): 0.0147 seconds


2023-11-04 00:56:23.599 python[39388:23371058] +[CATransaction synchronize] called within transaction
